In [30]:
import cv2
import torch
import numpy as np
from model import Dynamic_sparse_alignment_network
import os

# Preprocesamiento de la imagen
def preprocess_image(image):
    resized_image = cv2.resize(image, (256, 256))
    normalized_image = resized_image / 255.0
    tensor_image = torch.tensor(normalized_image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
    return tensor_image

# Extraer los 68 landmarks usando el mapeo proporcionado
def filter_68_landmarks(landmarks):
    """
    Extrae los 68 landmarks correspondientes al estándar MULTI-PIE
    usando el mapeo proporcionado.
    """
    mapping_indices = [
        0, 2, 4, 7, 9, 12, 14, 16, 18, 20, 22, 25, 27, 29, 31, 32,
        33, 34, 35, 36, 37, 42, 43, 44, 45, 46, 51, 52, 53, 54, 55,
        56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 71, 72, 73,
        75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95
    ]
    # Seleccionar la última etapa
    final_landmarks = landmarks[-1]  # Última etapa
    return final_landmarks[mapping_indices, :]

# Dibujar los 68 landmarks seleccionados
def draw_landmarks(image, landmarks):
    h, w, _ = image.shape
    for (x, y) in landmarks:
        x = int(x * w)
        y = int(y * h)
        cv2.circle(image, (x, y), 2, (0, 255, 0), 4)
    return image

# Guardar los landmarks en un archivo .txt en una sola línea
def save_landmarks_to_txt(landmarks, output_txt_path, image_width, image_height):
    """
    Guarda las coordenadas de los 68 landmarks en una sola línea en un archivo .txt,
    desnormalizando las coordenadas al tamaño original de la imagen.
    """
    # Desnormalizar los landmarks con respecto al tamaño de la imagen
    landmarks[:, 0] *= image_width
    landmarks[:, 1] *= image_height

    # Convertir a valores enteros
    landmarks = landmarks.flatten().astype(int)

    # Guardar en el archivo .txt
    with open(output_txt_path, 'w') as f:
        f.write(" ".join(map(str, landmarks)))  # Guardar en una sola línea
    print(f"Landmarks guardados en: {output_txt_path}")

# Inferencia del modelo
model_path = '/home/jocareher/Downloads/DSLPT_WFLW_6_layers.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Dynamic_sparse_alignment_network(num_point=98, d_model=256, trainable=False,
                                         return_interm_layers=False, nhead=8,
                                         feedforward_dim=1024, initial_path='/home/jocareher/Downloads/DSLPT/Config/init_98.npz',
                                         cfg=cfg)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Cargar la imagen
image_path = '/home/jocareher/Downloads/DSLPT/test.jpg'
output_image_path = '/home/jocareher/Downloads/DSLPT/resultado_landmarks_68_mapped.jpg'
output_txt_path = os.path.splitext(image_path)[0] + "_landmarks.txt"  # Cambiar extensión a .txt

image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f'No se pudo cargar la imagen en {image_path}')
input_tensor = preprocess_image(image).to(device)

# Obtener dimensiones de la imagen original
image_height, image_width, _ = image.shape

# Inferencia
with torch.no_grad():
    output_list, _, _, _ = model(input_tensor)
    landmarks = output_list[-1].squeeze().cpu().numpy()  # Tomar la última etapa
    landmarks_68 = filter_68_landmarks(landmarks)  # Extraer solo los 68 landmarks mapeados

print("Shape of 68 landmarks:", landmarks_68.shape)

# Dibujar y guardar la imagen con los landmarks
output_image = draw_landmarks(image, landmarks_68)
cv2.imwrite(output_image_path, output_image)
print(f"Imagen con 68 landmarks (MULTI-PIE) guardada en: {output_image_path}")

# Guardar los landmarks en un archivo .txt
save_landmarks_to_txt(landmarks_68, output_txt_path, image_width, image_height)






/tmp/ipykernel_7500/4011303703.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Shape of 68 landmarks: (67, 2)
Imagen con 68 landmarks (MULTI-PIE) guardada en: /home/jocareher/Downloads/DSLPT/resultado_landmarks_68_mapped.jpg
Landmarks guardados en: /home/jocareher/Downloads/DSLPT/test_landmarks.txt


In [29]:
landmarks_68

array([[0.18748502, 0.44528654],
       [0.19967188, 0.5250067 ],
       [0.21684518, 0.59900147],
       [0.24922614, 0.70296097],
       [0.2905845 , 0.7645465 ],
       [0.3723904 , 0.83415854],
       [0.4347434 , 0.8564329 ],
       [0.5098189 , 0.86540216],
       [0.5783227 , 0.8519188 ],
       [0.6462385 , 0.82336396],
       [0.7025631 , 0.7781673 ],
       [0.7639797 , 0.6881121 ],
       [0.7761883 , 0.61476076],
       [0.7831958 , 0.532358  ],
       [0.79275477, 0.45078713],
       [0.7958527 , 0.40981513],
       [0.22443517, 0.35752514],
       [0.26706412, 0.31549123],
       [0.31305024, 0.31036386],
       [0.3623707 , 0.31153414],
       [0.41346142, 0.31768158],
       [0.5397136 , 0.30778888],
       [0.58905673, 0.29746562],
       [0.6376335 , 0.29209602],
       [0.6880497 , 0.294482  ],
       [0.7362725 , 0.34083262],
       [0.48160094, 0.38464087],
       [0.48104492, 0.42887104],
       [0.48053518, 0.47552273],
       [0.48081717, 0.51827973],
       [0.